In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
gender = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
train['Fare'].isna().sum(), test['Fare'].isna().sum()

(0, 1)

In [5]:
train['Age'].isna().sum(), test['Age'].isna().sum()

(177, 86)

In [6]:
train['Embarked'].isna().sum(), test['Embarked'].isna().sum()

(2, 0)

In [7]:
train['age_null'] = train['Age'].isna().astype(int)
test['age_null'] = test['Age'].isna().astype(int)

def fill_age(row):
    if row['age_null'] == 1:
        return train[train['Pclass'] == row['Pclass']]['Age'].mean()
    else:
        return row['Age']
    
train['Age'] = train.apply(fill_age,axis=1)
test['Age'] = test.apply(fill_age,axis=1)

In [8]:
test['fare_null'] = test['Fare'].isna().astype(int)

def fill_fare(row):
    if row['fare_null'] == 1:
        return train[train['Pclass'] == row['Pclass']]['Fare'].mean()
    else:
        return row['Fare']
    
test['Fare'] = test.apply(fill_fare,axis=1)

In [9]:
train = train.dropna(subset=['Embarked'])

In [10]:
train['cabin_null'] = train['Cabin'].isna().astype(int)
test['cabin_null'] = test['Cabin'].isna().astype(int)

def fill_cabin(row):
    if row['cabin_null'] == 1:
        return "Yes"
    else:
        return "No"
    
train['Cabin'] = train.apply(fill_cabin,axis=1)
test['Cabin'] = test.apply(fill_cabin,axis=1)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [12]:
features = ["Pclass", "Sex", "SibSp", "Parch", "Age", "Fare", "Embarked",
            "Survived"]
model_test = pd.get_dummies(train[features], drop_first=True)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(model_test.drop(
    'Survived', axis=1), model_test['Survived'], test_size=0.2)

In [14]:
pgrid = {'LogisticRegression()': {
    'penalty': ['l1','l2'],
    'C': np.logspace(-5,5,12),
    'fit_intercept': [True,False],
    },
        'KNeighborsClassifier()': {
    'n_neighbors': range(3,20),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['manhattan', 'minkowski'],
    },
        'DecisionTreeClassifier()': {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [3,4,6,7,9],
    'min_samples_split': range(2,5),
    },
        'RandomForestClassifier()': {
            
    }}
scores = {'Model': [], 'Score': [], 'Params': []}
for ml_model in ['LogisticRegression()', 'KNeighborsClassifier()',
              'DecisionTreeClassifier()', 'RandomForestClassifier()']:
    model = eval(ml_model)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    pred_score = accuracy_score(Y_test, predictions)
    print(f"Using {ml_model}, we see a score of {pred_score}")
    scores['Model'].append(ml_model)
    scores['Score'].append(pred_score)
    scores['Params'].append(dict())
    rs = RandomizedSearchCV(model, pgrid[ml_model])
    rs.fit(X_train, Y_train)
    print(rs.best_params_)
    predictions = rs.predict(X_test)
    pred_score = accuracy_score(Y_test, predictions)
    print(f"Using {model}, we see a score of {pred_score}")
    scores['Model'].append(ml_model)
    scores['Score'].append(pred_score)
    scores['Params'].append(rs.best_params_)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Using LogisticRegression(), we see a score of 0.797752808988764


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{'penalty': 'l2', 'fit_intercept': True, 'C': 187.3817422860383}
Using LogisticRegression(), we see a score of 0.797752808988764
Using KNeighborsClassifier(), we see a score of 0.6685393258426966
{'weights': 'distance', 'n_neighbors': 14, 'metric': 'manhattan', 'algorithm': 'ball_tree'}
Using KNeighborsClassifier(), we see a score of 0.6629213483146067
Using DecisionTreeClassifier(), we see a score of 0.7921348314606742
{'splitter': 'best', 'min_samples_split': 2, 'max_depth': 6, 'criterion': 'entropy'}
Using DecisionTreeClassifier(), we see a score of 0.8426966292134831
Using RandomForestClassifier(), we see a score of 0.797752808988764


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{}
Using RandomForestClassifier(), we see a score of 0.7921348314606742


In [15]:
score_df = pd.DataFrame(scores)
max_score = score_df['Score'].max()
pick = score_df[score_df['Score'] == max_score].reset_index(drop=True)
model_sel = pick['Model'].iloc[0]
model_params = pick['Params'].iloc[0]
model_sel, model_params

('DecisionTreeClassifier()',
 {'splitter': 'best',
  'min_samples_split': 2,
  'max_depth': 6,
  'criterion': 'entropy'})

In [16]:
ml_dict = {
    'LogisticRegression()': LogisticRegression,
    'KNeighborsClassifier()': KNeighborsClassifier,
    'DecisionTreeClassifier()': DecisionTreeClassifier,
    'RandomForestClassifier()': RandomForestClassifier,
          }

def compile_function(model, params):
    param_str = ""
    for k,v in params.items():
        if isinstance(v, int):
            param_str += f'{k}={v}, '
        else:
            param_str += f'{k}="{v}", '
    param_str = param_str[:-2]
    return f'{model[:-1]}{param_str})'

y = train["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Age", "Fare", "Embarked"]
X = pd.get_dummies(train[features])
X_test = pd.get_dummies(test[features])

rs = RandomizedSearchCV(eval(model_sel), model_params)
x = compile_function(model_sel, model_params)
model = eval(x)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test.PassengerId,
                       'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [17]:
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
